In [1]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.4/720.4 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.6 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from agents import Agent, AsyncOpenAI, OpenAIChatCompletionsModel, RunConfig
from google.colab import userdata


gemini_api_key = userdata.get("GEMINI_NEW_KEY")
if gemini_api_key:
  print("Yes")
else:
  print("No")

client= AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

model = OpenAIChatCompletionsModel(
    openai_client=client,
    model="gemini-2.0-flash"
)

config= RunConfig(
    model_provider=client,
    model=model,
    tracing_disabled=True
)


Yes


In [5]:
import asyncio
from agents import Agent, Runner, handoff

spanish_agent= Agent(
    name="Alejandro",
    instructions="You will be answering questions asked in Spanish."
)

english_agent= Agent(
    name="John",
    instructions="You will be answering questions asked in English."
)


Customizing handoffs using the handoff() function:


In [10]:
from agents import Agent, handoff, RunContextWrapper

In [11]:
def on_handoff(agent: Agent, context: RunContextWrapper):
    agent_name = agent.name
    print("-------------------------")
    print(f"Handoff to {agent_name}")
    print("-------------------------")


In [12]:
triage_agent= Agent(
    name="Manager",
    instructions="You need to delegate the request to the appropriate agent depending on the language.",
    handoffs= [handoff(spanish_agent, on_handoff=lambda ctx: on_handoff(spanish_agent, ctx)),
            handoff(english_agent, on_handoff=lambda ctx: on_handoff(english_agent, ctx))]
)

async def main(input: str):
    result = await Runner.run(triage_agent, input=input, run_config=config)
    print(result.final_output)

In [13]:
asyncio.run(main("when was the earth came into existence?"))

-------------------------
Handoff to John
-------------------------
The Earth is estimated to have formed around 4.54 billion years ago.



In [14]:
asyncio.run(main("Donde esta Santa Clara?"))

-------------------------
Handoff to Alejandro
-------------------------
Hay muchas ciudades llamadas Santa Clara.  ¿Podrías ser más específico? ¿Te refieres a Santa Clara, Cuba, o a Santa Clara, California, en los Estados Unidos, o a alguna otra Santa Clara?



In [15]:
asyncio.run(main("La Santa Clara que ha mencionada en \"The Call of the Wild."))

-------------------------
Handoff to Alejandro
-------------------------
En "La Llamada de lo Salvaje" (The Call of the Wild), "Santa Clara" se refiere a la **ciudad de Santa Clara, California.**  Se menciona como parte del entorno civilizado del que Buck es arrancado al principio de la historia.  Él vivía una vida cómoda y mimada como perro doméstico en una finca en el valle de Santa Clara, antes de ser secuestrado y vendido como perro de trineo en el Klondike.

